In [49]:
import torch
import torch.nn as nn
import pandas as pd

In [50]:
raw_df = pd.read_csv('./kaggle-mnist/Data/train.csv')
#feature
feature = raw_df.drop(['label'], axis=1).values
#https://medium.com/hccuse/pandas%E5%AD%B8%E7%BF%92%E7%AD%86%E8%A8%98-%E8%BB%B8%E5%90%91%E7%90%86%E8%A7%A3-1084c6c5e873

#label
label = raw_df['label'].values

In [51]:
print(len(label), len(feature))

42000 42000


In [52]:
train_feature = feature[:int(len(feature)*0.8)]
train_label = label[:int(len(label)*0.8)]

test_feature = feature[:int(len(feature)*0.2)]
test_label = label[:int(len(label)*0.2)]
print(len(train_feature), len(train_label), len(test_feature), len(test_label))

train_feature = torch.tensor(train_feature).to(torch.float).cuda()
train_label = torch.tensor(train_label).cuda()
test_feature = torch.tensor(test_feature).to(torch.float).cuda()
test_label = torch.tensor(test_label).cuda()

33600 33600 8400 8400


In [53]:
model = nn.Sequential(
    nn.Linear(784, 444),#wx+b
    nn.ReLU(),
    nn.Linear(444, 512),
    nn.ReLU(),
    nn.Linear(512, 512),
    nn.ReLU(),
    nn.Linear(512, 10),
#     nn.Softmax()#CrossEntropyLoss時會默認算一次softmax
).cuda()#也可以放在這

#將資料放入gpu ram
# model = model.cuda()
print(model)

Sequential(
  (0): Linear(in_features=784, out_features=444, bias=True)
  (1): ReLU()
  (2): Linear(in_features=444, out_features=512, bias=True)
  (3): ReLU()
  (4): Linear(in_features=512, out_features=512, bias=True)
  (5): ReLU()
  (6): Linear(in_features=512, out_features=10, bias=True)
)


In [ ]:
# data = torch.rand(1,784)
# pred = model(data)
# print(pred)

In [55]:
lossfunction = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(params=model.parameters(), lr=0.0001)

In [56]:
for i in range(100):
    #first
    optimizer.zero_grad()
    predict_train = model(train_feature)
    result_train = torch.argmax(predict_train, axis=1)
    train_acc = torch.mean((result_train == train_label).to(torch.float))
#     print(train_acc)
    loss_train = lossfunction(predict_train, train_label)
    loss_train.backward()#先傳播再梯度下降
    optimizer.step()
#     print(loss.item())

    print(f'train loss:{loss_train.item()} train acc:{train_acc}')
    
    #second on test
    optimizer.zero_grad()
    predict_test = model(test_feature)
    result_test = torch.argmax(predict_test, axis=1)
    test_acc = torch.mean((result_test == test_label).to(torch.float))
    loss_test = lossfunction(predict_test, test_label)
    print(f'test loss:{loss_test.item()} test acc:{test_acc}')

train loss:5.744922161102295 train acc:0.11937499791383743
test loss:3.9235002994537354 test acc:0.1522618979215622
train loss:3.883194923400879 train acc:0.15160714089870453
test loss:3.0466742515563965 test acc:0.25999999046325684
train loss:3.039297580718994 train acc:0.2579761743545532
test loss:2.6159260272979736 test acc:0.3372619152069092
train loss:2.6265716552734375 train acc:0.3311011791229248
test loss:2.310678243637085 test acc:0.3963095247745514
train loss:2.3321468830108643 train acc:0.39175593852996826
test loss:1.9959758520126343 test acc:0.46857142448425293
train loss:2.0252480506896973 train acc:0.4642857015132904
test loss:1.7283117771148682 test acc:0.5248809456825256
train loss:1.7613660097122192 train acc:0.5205357074737549
test loss:1.4713302850723267 test acc:0.5682142972946167
train loss:1.5019633769989014 train acc:0.5647023916244507
test loss:1.2066363096237183 test acc:0.6279761791229248
train loss:1.2322171926498413 train acc:0.6230059266090393
test loss:0.

In [42]:
torch.save(model.state_dict(), './model.pt')#w and b

# 載入模型參數

In [45]:
params = torch.load('./model.pt')

model.load_state_dict(params)#要有之前的model結構，我們只有存參數

<All keys matched successfully>

In [46]:
new_test_data = test_feature[100:111]
new_test_label = test_label[100:111]

In [48]:
predict_test = model(new_test_data)
result_test = torch.argmax(predict_test, axis=1)
print("label")
print(new_test_label)

print("predict")
print(result_test)

label
tensor([9, 2, 7, 7, 2, 8, 8, 5, 0, 6, 0])
predict
tensor([9, 2, 7, 7, 2, 8, 8, 5, 0, 6, 0])


# 資料與模型可以放在GPU來加速